# for_mnm_matching_algorithm
> KNN 알고리즘을 mnm에 적합하게 직접 구현


> mbti까지모두 적용


### 라이브러리 불러오기

In [233]:
import pandas as pd
import numpy as np

### 설문조사 데이터 불러오기

In [234]:
df = pd.read_excel("하우스 메이트 설문결과.xlsx")
df.head()

,TIMESTAMP,1. 성별,2. 나이,3. mbti,4. 흡연 유무,4-1. 원하는 메이트의 흡연여부,5. 반려동물과 같이 살 예정이다.,5-1. 반려동물을 키울 경우 어떤 동물인가?,5-2. 메이트의 반려동물 중 허용가능한 것은?,6. 난방/냉방 관련 문제,"7. 벌레를 잡을 수 있는가? (바퀴벌레, 돈벌레, 지네 정도)",7-1. 원하는 메이트,8.집에서 밥을 먹을 경우의 식사 스타일,8-1. 메이트가 집에서 요리하는 게 불편한가요? (냄새 문제 등),9. 선호하는 식사유형,10. 메이트와 물건을 공유해도 괜찮습니까?,11. 방내에서의 소음문제,12. 음주 문제,13. 정리정돈을 신경쓰는 정도,14. 지인 출입
0,2021-10-20 11:29:52,여자,24 ~ 26,infp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지,절대 못잡는다,NaN,벌레를 잡을 수 있어야만 한다,직접 요리,NaN,그렇다,NaN,NaN,소음은 없었으면 좋겠다.,2,누구든 데려오면 안된다.
1,2021-10-22 01:16:34,남자,27 ~ 29,intp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지,더위를 많이 타서 에어컨을 일찍부터 튼다\t여름에 항상 에어컨을 켜놓는다\t과한 난...,시키면 잡을 수 있다,벌레를 못잡아도 상관없다,직접 요리,NaN,혼자 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,신경쓰지 않는다.,메이트가 술주정만 하지 않으면 된다,내 공간만 아니면 신경쓰이지 않는다,미리 말만 하면 지인이 출입해도 상관없다.
2,2021-10-22 01:29:18,여자,20 ~ 23,esfp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지\t조류,더위를 많이 타서 에어컨을 일찍부터 튼다\t추위를 많이 타서 난방을 일찍부터 튼다\...,절대 못잡는다,벌레를 잡을 수 있어야만 한다,"외부 음식(포장, 배달)",NaN,혼자 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,10분 이상의 동영상/음악 소리는 신경쓰인다\t지속적인 알람소리는 신경쓰인다,메이트가 술주정만 하지 않으면 된다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
3,2021-10-22 01:31:02,남자,20 ~ 23,모른다,비흡연자,비흡연자,아니다,NaN,어떠한 동물도 안된다.,NaN,잡는데 거리낌이 없다,벌레를 못잡아도 상관없다,직접 요리,NaN,같이 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,신경쓰지 않는다.,메이트의 술 문제는 신경쓰지 않는다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
4,2021-10-22 01:33:35,남자,20 ~ 23,intp,비흡연자,비흡연자,아니다,NaN,어떠한 동물도 안된다.,과한 난방은 답답해하는 편이다,시키면 잡을 수 있다,벌레를 못잡아도 상관없다,직접 요리,NaN,혼자 먹는 걸 선호한다,괜찮지 않다(안된다),지속적인 알람소리는 신경쓰인다,메이트가 술 냄새를 풍기는 것은 싫다\t메이트가 술을 많이 마신 것 같은 모습을 보...,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.


In [235]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 20 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   TIMESTAMP                              72 non-null     datetime64[ns]
 1   1. 성별                                  72 non-null     object        
 2   2. 나이                                  72 non-null     object        
 3   3. mbti                                72 non-null     object        
 4   4. 흡연 유무                               72 non-null     object        
 5   4-1. 원하는 메이트의 흡연여부                     72 non-null     object        
 6   5. 반려동물과 같이 살 예정이다.                    72 non-null     object        
 7   5-1. 반려동물을 키울 경우 어떤 동물인가?              15 non-null     object        
 8   5-2. 메이트의 반려동물 중 허용가능한 것은?             72 non-null     object        
 9   6. 난방/냉방 관련 문제                         71 non-null     object      

### 컬럼 이름 변경
- 컬럼 이름을 데이터베이스 테이블 명세서에 맞춰 변경한다.
- time 컬럼을 제외하면 총 19개의 컬럼을 가진다.

In [236]:
df.columns = ['time', 'sex', 'age', 'mbti', 'user_smoking', 'mate_smoking', 'user_pet', 'user_pet_category', 'mate_pet_category', 
              'air', 'user_bug_killer', 'mate_bug_killer', 'user_cooking', 'mate_cooking', 'eat_together', 'share_item', 'noise', 'mate_alcohol', 
              'mate_clean', 'permission_to_enter']
df.head()

,time,sex,age,mbti,user_smoking,mate_smoking,user_pet,user_pet_category,mate_pet_category,air,user_bug_killer,mate_bug_killer,user_cooking,mate_cooking,eat_together,share_item,noise,mate_alcohol,mate_clean,permission_to_enter
0,2021-10-20 11:29:52,여자,24 ~ 26,infp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지,절대 못잡는다,NaN,벌레를 잡을 수 있어야만 한다,직접 요리,NaN,그렇다,NaN,NaN,소음은 없었으면 좋겠다.,2,누구든 데려오면 안된다.
1,2021-10-22 01:16:34,남자,27 ~ 29,intp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지,더위를 많이 타서 에어컨을 일찍부터 튼다\t여름에 항상 에어컨을 켜놓는다\t과한 난...,시키면 잡을 수 있다,벌레를 못잡아도 상관없다,직접 요리,NaN,혼자 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,신경쓰지 않는다.,메이트가 술주정만 하지 않으면 된다,내 공간만 아니면 신경쓰이지 않는다,미리 말만 하면 지인이 출입해도 상관없다.
2,2021-10-22 01:29:18,여자,20 ~ 23,esfp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지\t조류,더위를 많이 타서 에어컨을 일찍부터 튼다\t추위를 많이 타서 난방을 일찍부터 튼다\...,절대 못잡는다,벌레를 잡을 수 있어야만 한다,"외부 음식(포장, 배달)",NaN,혼자 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,10분 이상의 동영상/음악 소리는 신경쓰인다\t지속적인 알람소리는 신경쓰인다,메이트가 술주정만 하지 않으면 된다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
3,2021-10-22 01:31:02,남자,20 ~ 23,모른다,비흡연자,비흡연자,아니다,NaN,어떠한 동물도 안된다.,NaN,잡는데 거리낌이 없다,벌레를 못잡아도 상관없다,직접 요리,NaN,같이 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,신경쓰지 않는다.,메이트의 술 문제는 신경쓰지 않는다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
4,2021-10-22 01:33:35,남자,20 ~ 23,intp,비흡연자,비흡연자,아니다,NaN,어떠한 동물도 안된다.,과한 난방은 답답해하는 편이다,시키면 잡을 수 있다,벌레를 못잡아도 상관없다,직접 요리,NaN,혼자 먹는 걸 선호한다,괜찮지 않다(안된다),지속적인 알람소리는 신경쓰인다,메이트가 술 냄새를 풍기는 것은 싫다\t메이트가 술을 많이 마신 것 같은 모습을 보...,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.


### 컬럼 설명
- sex : 여자, 남자
- age : 10대, 20 ~ 23, 24 ~ 26, 27 ~ 29, 30대, 40대
- mbti : 따로 처리 필요
- user_smoking : 흡연자, 비흡연자
- mate_smoking : 흡연자, 비흡연자, 상관없음
  - 상관없음의 경우 해당 컬럼을 사용하지 않는 경우와 중간값을 주는 두가지의 방법을 사용할 수 있다.
- user_pet : 그렇다, 아니다
- user_pet_category : 강아지, 고양이, '파충류, 어류', 설치류, 조류, '기타'
  - 
- mate_pet_category : 어떠한 동물도 안된다., 어떤 동물이든 괜찮다., 고양이, 강아지, '파충류, 어류', 설치류, 조류
  - 어떠한 동물도 안된다. : user_pet == '그렇다' 일 경우를 제외한다.
  - 어떤 동물이든 괜찮다. : 
- air : 더위를 많이 타서 에어컨을 일찍부터 튼다, 여름에 항상 에어컨을 켜놓는다 = air_like_airconditionner / 에어컨을 틀고 못잔다 = air_night_airconditioner / 추위를 많이 타서 난방을 일찍부터 튼다, 과한 난방은 답답해하는 편이다 = air_like_heater / 해당하는 상황 없음 = air = none
- user_bug_killer : 잡는데 거리낌이 없다, 시키면 잡을 수 있다, 절대 못잡는다
- mate_bug_killer : 벌레를 잡을 수 있어야 한다, 벌레를 못잡아도 상관없다
- user_cooking : 직접 요리, 외부 음식(포장, 배달)
- mate_cooking : 불편하다, 신경쓰지 않는다(불편하지 않다)
- eat_together : 혼자 먹는 걸 선호한다, 같이 먹는 걸 선호한다
- shre_item : 말 안하고 사용해도 괜찮다, 사용하기 전 허락을 받으면 괜찮다, 괜찮지 않다(안된다)
- noise : 신경쓰지 않는다. = noise_none / 10분 이상의 말소리(통화/게임)은 신경쓰인다, 10분 이전의 말소리(통화/게임)도 신경쓰인다 = noise_talking / 10분 이상의 동영상/음악 소리는 신경쓰인다, 10분 이전의 동영상/음악 소리도 신경쓰인다 = noise_music / 지속적인 알람소리는 신경쓰인다 = noise_alarm
- mate_alcohol : 메이트의 술 문제는 신경쓰지 않는다, 메이트가 술 냄새를 풍기는 것은 싫다, 메이트가 술을 많이 마신 것 같은 모습을 보이면 싫다, 메이트가 술주정만 하지 않으면 된다
- mate_clean : 내 공간만 아니면 신경쓰이지 않는다, 공유하는 공간의 경우 신경쓰인다, 메이트의 공간까지 신경쓰인다
- permission_to_enter : 본인이 없을 때 말없이 지인이 잠깐 출입해도 상관없다., 미리 말만 하면 지인이 출입해도 상관없다., 누구든 데려오면 안된다.

In [237]:
df.isnull().sum()

time                    0
sex                     0
age                     0
mbti                    0
user_smoking            0
mate_smoking            0
user_pet                0
user_pet_category      57
mate_pet_category       0
air                     1
user_bug_killer         1
mate_bug_killer         0
user_cooking            0
mate_cooking            8
eat_together            0
share_item              1
noise                   2
mate_alcohol            1
mate_clean              0
permission_to_enter     0
dtype: int64

### 컬럼별 데이터 전처리
- 컬럼 분리
- 데이터를 0~1사이의 숫자로 변환

- sex

In [238]:
print(df['sex'].value_counts())
print("NaN : ", df['sex'].isnull().sum())

여자    54
남자    18
Name: sex, dtype: int64
NaN :  0


In [239]:
df['sex'] = df['sex'].apply(lambda x : 1 if str(x) == '여자' else 0)
print(df['sex'].value_counts())
print("NaN : ", df['sex'].isnull().sum())

1    54
0    18
Name: sex, dtype: int64
NaN :  0


- age

In [240]:
print(df['age'].value_counts())
print("NaN : ", df['age'].isnull().sum())

24 ~ 26    32
20 ~ 23    26
27 ~ 29     7
30대         4
10대         3
Name: age, dtype: int64
NaN :  0


In [241]:
def func(x):
  if '40대 이후' == str(x):
    return 1
  elif '30대' == str(x):
    return 0.8
  elif '27 ~ 29' == str(x):
    return 0.6
  elif '24 ~ 26' == str(x):
    return 0.4
  elif '20 ~ 23' == str(x):
    return 0.2
  elif '10대' == str(x):
    return 0
  else:
    return np.nan

df['age'] = df['age'].apply(lambda x : func(x))
print(df['age'].value_counts())
print("NaN : ", df['age'].isnull().sum())

0.4    32
0.2    26
0.6     7
0.8     4
0.0     3
Name: age, dtype: int64
NaN :  0


- user_smoking : 흡연자, 비흡연자
- mate_smoking : 흡연자, 비흡연자, 상관없음
  - 상관없음의 경우 해당 컬럼을 사용하지 않는 경우와 중간값을 주는 두가지의 방법을 사용할 수 있다.

In [242]:
print(df['user_smoking'].value_counts())
print("NaN : ", df['user_smoking'].isnull().sum())

비흡연자    64
흡연자      8
Name: user_smoking, dtype: int64
NaN :  0


In [243]:
df['user_smoking'] = df['user_smoking'].apply(lambda x : 1 if str(x) == '흡연자' else 0)
print(df['user_smoking'].value_counts())
print("NaN : ", df['user_smoking'].isnull().sum())

0    64
1     8
Name: user_smoking, dtype: int64
NaN :  0


In [244]:
print(df['mate_smoking'].value_counts())
print("NaN : ", df['mate_smoking'].isnull().sum())

비흡연자    58
상관없음    11
흡연자      3
Name: mate_smoking, dtype: int64
NaN :  0


In [245]:
def func(x):
  if '흡연자' == str(x):
    return 1
  elif '상관없음'== str(x):
    return 0.5
  elif '비흡연자' == str(x):
    return 0
  else:
    return np.nan

df['mate_smoking'] = df['mate_smoking'].apply(lambda x : func(x))
print(df['mate_smoking'].value_counts())
print("NaN : ", df['mate_smoking'].isnull().sum())

0.0    58
0.5    11
1.0     3
Name: mate_smoking, dtype: int64
NaN :  0


- user_pet : 그렇다, 아니다
- user_pet_category : 강아지, 고양이, '파충류, 어류', 설치류, 조류, '기타'
  - 
- mate_pet_category : 어떠한 동물도 안된다., 어떤 동물이든 괜찮다., 고양이, 강아지, '파충류, 어류', 설치류, 조류
  - 어떠한 동물도 안된다. : user_pet == '그렇다' 일 경우를 제외한다.
  - 어떤 동물이든 괜찮다. : 

In [246]:
print(df['user_pet'].value_counts())
print("NaN : ", df['user_pet'].isnull().sum())

아니다    60
그렇다    12
Name: user_pet, dtype: int64
NaN :  0


In [247]:
df['user_pet'] = df['user_pet'].apply(lambda x : 1 if str(x) == '그렇다' else 0)
print(df['user_pet'].value_counts())
print("NaN : ", df['user_pet'].isnull().sum())

0    60
1    12
Name: user_pet, dtype: int64
NaN :  0


- user_pet_category 컬럼을 분리해야한다.

In [248]:
print(df['user_pet_category'].value_counts())
print("NaN : ", df['user_pet_category'].isnull().sum())

강아지              7
고양이              3
강아지\t고양이         3
강아지\t고양이\t해파리    1
동물털 알러지 있음       1
Name: user_pet_category, dtype: int64
NaN :  57


In [249]:
df['user_pet_dog'] = df['user_pet_category'].apply(lambda x : 1 if '강아지' in str(x) else 0)
print(df['user_pet_dog'].value_counts())
print("NaN : ", df['user_pet_dog'].isnull().sum())

0    61
1    11
Name: user_pet_dog, dtype: int64
NaN :  0


In [250]:
df['user_pet_cat'] = df['user_pet_category'].apply(lambda x : 1 if '고양이' in str(x) else 0)
print(df['user_pet_cat'].value_counts())
print("NaN : ", df['user_pet_cat'].isnull().sum())

0    65
1     7
Name: user_pet_cat, dtype: int64
NaN :  0


In [251]:
df['user_pet_reptie_fish'] = df['user_pet_category'].apply(lambda x : 1 if '파충류, 어류' in str(x) else 0)
print(df['user_pet_reptie_fish'].value_counts())
print("NaN : ", df['user_pet_reptie_fish'].isnull().sum())

0    72
Name: user_pet_reptie_fish, dtype: int64
NaN :  0


In [252]:
df['user_pet_rodent'] = df['user_pet_category'].apply(lambda x : 1 if '설치류' in str(x) else 0)
print(df['user_pet_rodent'].value_counts())
print("NaN : ", df['user_pet_rodent'].isnull().sum())

0    72
Name: user_pet_rodent, dtype: int64
NaN :  0


In [253]:
df['user_pet_bird'] = df['user_pet_category'].apply(lambda x : 1 if '조류' in str(x) else 0)
print(df['user_pet_bird'].value_counts())
print("NaN : ", df['user_pet_bird'].isnull().sum())

0    72
Name: user_pet_bird, dtype: int64
NaN :  0


In [254]:
# user_pet_category 삭제
df = df.drop(columns='user_pet_category')
df.columns

Index(['time', 'sex', 'age', 'mbti', 'user_smoking', 'mate_smoking',
       'user_pet', 'mate_pet_category', 'air', 'user_bug_killer',
       'mate_bug_killer', 'user_cooking', 'mate_cooking', 'eat_together',
       'share_item', 'noise', 'mate_alcohol', 'mate_clean',
       'permission_to_enter', 'user_pet_dog', 'user_pet_cat',
       'user_pet_reptie_fish', 'user_pet_rodent', 'user_pet_bird'],
      dtype='object')

- mate_pet_category 컬럼을 분리해야한다.

In [255]:
print(df['mate_pet_category'].value_counts())
print("NaN : ", df['mate_pet_category'].isnull().sum())

어떠한 동물도 안된다.                  21
고양이\t강아지                      20
강아지                           11
고양이                            5
어떤 동물이든 괜찮다.                   3
고양이\t강아지\t파충류, 어류              3
파충류, 어류                        2
고양이\t강아지\t조류                   2
어떠한 동물도 안된다.\t어떤 동물이든 괜찮다.     1
고양이\t강아지\t파충류, 어류\t설치류         1
고양이\t파충류, 어류\t설치류\t조류          1
고양이\t강아지\t파충류, 어류\t조류          1
고양이\t강아지\t설치류                  1
Name: mate_pet_category, dtype: int64
NaN :  0


In [256]:
def func(x):
  if '어떠한 동물도 안된다.' in str(x):
    return 1
  elif '어떤 동물이든 괜찮다.' in str(x):
    return 0
  else:
    return 0.5 # NaN, 그 외

df['mate_pet'] = df['mate_pet_category'].apply(lambda x : func(x))
print(df['mate_pet'].value_counts())
print("NaN : ", df['mate_pet'].isnull().sum())

0.5    47
1.0    22
0.0     3
Name: mate_pet, dtype: int64
NaN :  0


In [257]:
df['mate_pet_dog'] = df['mate_pet_category'].apply(lambda x : 0.5 if '강아지' in str(x) or '어떤 동물이든 괜찮다.' in str(x) else 0)
print(df['mate_pet_dog'].value_counts())
print("NaN : ", df['mate_pet_dog'].isnull().sum())

0.5    43
0.0    29
Name: mate_pet_dog, dtype: int64
NaN :  0


In [258]:
df['mate_pet_cat'] = df['mate_pet_category'].apply(lambda x : 0.5 if '고양이' in str(x) or '어떤 동물이든 괜찮다.' in str(x) else 0)
print(df['mate_pet_cat'].value_counts())
print("NaN : ", df['mate_pet_cat'].isnull().sum())

0.5    38
0.0    34
Name: mate_pet_cat, dtype: int64
NaN :  0


In [259]:
df['mate_pet_reptie_fish'] = df['mate_pet_category'].apply(lambda x : 0.5 if '파충류, 어류' in str(x) or '어떤 동물이든 괜찮다.' in str(x) else 0)
print(df['mate_pet_reptie_fish'].value_counts())
print("NaN : ", df['mate_pet_reptie_fish'].isnull().sum())

0.0    60
0.5    12
Name: mate_pet_reptie_fish, dtype: int64
NaN :  0


In [260]:
df['mate_pet_rodent'] = df['mate_pet_category'].apply(lambda x : 0.5 if '설치류' in str(x) or '어떤 동물이든 괜찮다.' in str(x) else 0)
print(df['mate_pet_rodent'].value_counts())
print("NaN : ", df['mate_pet_rodent'].isnull().sum())

0.0    65
0.5     7
Name: mate_pet_rodent, dtype: int64
NaN :  0


In [261]:
df['mate_pet_bird'] = df['mate_pet_category'].apply(lambda x : 0.5 if '조류' in str(x) or '어떤 동물이든 괜찮다.' in str(x) else 0)
print(df['mate_pet_bird'].value_counts())
print("NaN : ", df['mate_pet_bird'].isnull().sum())

0.0    64
0.5     8
Name: mate_pet_bird, dtype: int64
NaN :  0


In [262]:
# mate_pet_category 삭제
df = df.drop(columns='mate_pet_category')
df.columns

Index(['time', 'sex', 'age', 'mbti', 'user_smoking', 'mate_smoking',
       'user_pet', 'air', 'user_bug_killer', 'mate_bug_killer', 'user_cooking',
       'mate_cooking', 'eat_together', 'share_item', 'noise', 'mate_alcohol',
       'mate_clean', 'permission_to_enter', 'user_pet_dog', 'user_pet_cat',
       'user_pet_reptie_fish', 'user_pet_rodent', 'user_pet_bird', 'mate_pet',
       'mate_pet_dog', 'mate_pet_cat', 'mate_pet_reptie_fish',
       'mate_pet_rodent', 'mate_pet_bird'],
      dtype='object')

- air 컬럼을 분리해야한다.
  - 더위를 많이 타서 에어컨을 일찍부터 튼다, 여름에 항상 에어컨을 켜놓는다 = air_like_airconditionner / 에어컨을 틀고 못잔다 = air_night_airconditioner / 추위를 많이 타서 난방을 일찍부터 튼다, 과한 난방은 답답해하는 편이다 = air_like_heater / 해당하는 상황 없음 = air = none

In [263]:
df['air'].value_counts()

과한 난방은 답답해하는 편이다                                                                     12
해당하는 상황 없음                                                                            8
더위를 많이 타서 에어컨을 일찍부터 튼다\t과한 난방은 답답해하는 편이다                                              8
더위를 많이 타서 에어컨을 일찍부터 튼다\t여름에 항상 에어컨을 켜놓는다\t과한 난방은 답답해하는 편이다                            6
추위를 많이 타서 난방을 일찍부터 튼다                                                                 6
더위를 많이 타서 에어컨을 일찍부터 튼다                                                                5
더위를 많이 타서 에어컨을 일찍부터 튼다\t추위를 많이 타서 난방을 일찍부터 튼다                                         4
여름에 항상 에어컨을 켜놓는다\t과한 난방은 답답해하는 편이다                                                    4
더위를 많이 타서 에어컨을 일찍부터 튼다\t추위를 많이 타서 난방을 일찍부터 튼다\t과한 난방은 답답해하는 편이다                       3
에어컨을 틀고 못잔다\t과한 난방은 답답해하는 편이다                                                         3
추위를 많이 타서 난방을 일찍부터 튼다\t과한 난방은 답답해하는 편이다                                               3
여름에 항상 에어컨을 켜놓는다                

In [264]:
# NaN -> '해당하는 상황 없음'
df['air'] = df['air'].fillna('해당하는 상황 없음')

In [265]:
df['air_like_airconditioner'] = df['air'].apply(lambda x : 1 if '더위를 많이 타서 에어컨을 일찍부터 튼다' in str(x) or '여름에 항상 에어컨을 켜놓는다' in str(x) else 0)
df['air_like_airconditioner'].value_counts()

0    37
1    35
Name: air_like_airconditioner, dtype: int64

In [266]:
df['air_like_airconditioner'].isnull().sum()

0

In [267]:
df['air_night_airconditioner'] = df['air'].apply(lambda x : 1 if '에어컨을 틀고 못잔다' in str(x) else 0)
df['air_night_airconditioner'].value_counts()

0    66
1     6
Name: air_night_airconditioner, dtype: int64

In [268]:
df['air_night_airconditioner'].isnull().sum()

0

In [269]:
def func(x):
  if '추위를 많이 타서 난방을 일찍부터 튼다' in str(x):
    return 1
  elif '과한 난방은 답답해하는 편이다'in str(x):
    return 0
  else:
    return 0.5 # 난방에 대해 신경쓰지 않는다고 판단

df['air_like_heater'] = df['air'].apply(lambda x : func(x))
df['air_like_heater'].value_counts()

0.0    33
1.0    20
0.5    19
Name: air_like_heater, dtype: int64

In [270]:
df['air_like_heater'].isnull().sum()

0

In [271]:
# air 삭제
df = df.drop(columns='air')
df.columns

Index(['time', 'sex', 'age', 'mbti', 'user_smoking', 'mate_smoking',
       'user_pet', 'user_bug_killer', 'mate_bug_killer', 'user_cooking',
       'mate_cooking', 'eat_together', 'share_item', 'noise', 'mate_alcohol',
       'mate_clean', 'permission_to_enter', 'user_pet_dog', 'user_pet_cat',
       'user_pet_reptie_fish', 'user_pet_rodent', 'user_pet_bird', 'mate_pet',
       'mate_pet_dog', 'mate_pet_cat', 'mate_pet_reptie_fish',
       'mate_pet_rodent', 'mate_pet_bird', 'air_like_airconditioner',
       'air_night_airconditioner', 'air_like_heater'],
      dtype='object')

- noise 컬럼을 분리해야한다.
  - 신경쓰지 않는다. = noise_none / 10분 이상의 말소리(통화/게임)은 신경쓰인다, 10분 이전의 말소리(통화/게임)도 신경쓰인다 = noise_talking / 10분 이상의 동영상/음악 소리는 신경쓰인다, 10분 이전의 동영상/음악 소리도 신경쓰인다 = noise_music / 지속적인 알람소리는 신경쓰인다 = noise_alarm

In [272]:
# NaN -> '신경쓰지 않는다.'
df['noise'] = df['noise'].fillna('신경쓰지 않는다.')

In [273]:
def func(x):
  if '10분 이전의 말소리(통화/게임)도 신경쓰인다' in str(x):
    return 1
  elif '10분 이상의 말소리(통화/게임)은 신경쓰인다'in str(x):
    return 0.5
  else:
    return 0

df['noise_talking'] = df['noise'].apply(lambda x : func(x))
df['noise_talking'].value_counts()

0.0    43
0.5    24
1.0     5
Name: noise_talking, dtype: int64

In [274]:
# 결측치 확인
df['noise_talking'].isnull().sum()

0

In [275]:
def func(x):
  if '10분 이전의 동영상/음악 소리도 신경쓰인다' in str(x):
    return 1
  elif '10분 이상의 동영상/음악 소리는 신경쓰인다'in str(x):
    return 0.5
  else:
    return 0

df['noise_music'] = df['noise'].apply(lambda x : func(x))
df['noise_music'].value_counts()

0.0    43
0.5    24
1.0     5
Name: noise_music, dtype: int64

In [276]:
df['noise_music'].isnull().sum()

0

In [277]:
df['noise_alarm'] = df['noise'].apply(lambda x : 1 if '지속적인 알람소리는 신경쓰인다' in str(x) else 0)
df['noise_alarm'].value_counts()

1    46
0    26
Name: noise_alarm, dtype: int64

In [278]:
df['noise_alarm'].isnull().sum()

0

In [279]:
# noise 삭제
df = df.drop(columns='noise')
df.columns

Index(['time', 'sex', 'age', 'mbti', 'user_smoking', 'mate_smoking',
       'user_pet', 'user_bug_killer', 'mate_bug_killer', 'user_cooking',
       'mate_cooking', 'eat_together', 'share_item', 'mate_alcohol',
       'mate_clean', 'permission_to_enter', 'user_pet_dog', 'user_pet_cat',
       'user_pet_reptie_fish', 'user_pet_rodent', 'user_pet_bird', 'mate_pet',
       'mate_pet_dog', 'mate_pet_cat', 'mate_pet_reptie_fish',
       'mate_pet_rodent', 'mate_pet_bird', 'air_like_airconditioner',
       'air_night_airconditioner', 'air_like_heater', 'noise_talking',
       'noise_music', 'noise_alarm'],
      dtype='object')

- user_bug_killer : 잡는데 거리낌이 없다, 시키면 잡을 수 있다, 절대 못잡는다
- mate_bug_killer : 벌레를 잡을 수 있어야 한다, 벌레를 못잡아도 상관없다

In [280]:
df['user_bug_killer'].value_counts()

시키면 잡을 수 있다    29
절대 못잡는다        23
잡는데 거리낌이 없다    19
Name: user_bug_killer, dtype: int64

In [281]:
df['user_bug_killer'].isnull().sum()

1

In [282]:
def func(x):
  if '잡는데 거리낌이 없다' in str(x):
    return 1
  elif '시키면 잡을 수 있다'in str(x):
    return 0.75
  elif '절대 못잡는다' in str(x):
    return 0
  else:
    return np.nan

df['user_bug_killer'] = df['user_bug_killer'].apply(lambda x : func(x))
df['user_bug_killer'].value_counts()

0.75    29
0.00    23
1.00    19
Name: user_bug_killer, dtype: int64

In [283]:
# mean으로 결측치 채움
df['user_bug_killer'] = df['user_bug_killer'].fillna(df['user_bug_killer'].mean())
df['user_bug_killer'].value_counts()

0.750000    29
0.000000    23
1.000000    19
0.573944     1
Name: user_bug_killer, dtype: int64

In [284]:
df['mate_bug_killer'].value_counts()

벌레를 못잡아도 상관없다       40
벌레를 잡을 수 있어야만 한다    32
Name: mate_bug_killer, dtype: int64

In [285]:
df['mate_bug_killer'].isnull().sum()

0

In [286]:
df['mate_bug_killer'] = df['mate_bug_killer'].apply(lambda x : 1 if '벌레를 잡을 수 있어야만 한다' in str(x) else 0)
df['mate_bug_killer'].value_counts()

0    40
1    32
Name: mate_bug_killer, dtype: int64

- user_cooking : 직접 요리, 외부 음식(포장, 배달)
- mate_cooking : 불편하다, 신경쓰지 않는다(불편하지 않다)


In [287]:
df['user_cooking'].value_counts()

직접 요리            42
외부 음식(포장, 배달)    30
Name: user_cooking, dtype: int64

In [288]:
df['user_cooking'].isnull().sum()

0

In [289]:
df['user_cooking'] = df['user_cooking'].apply(lambda x : 1 if '외부 음식(포장, 배달)' in str(x) else 0)
df['user_cooking'].value_counts()

0    42
1    30
Name: user_cooking, dtype: int64

In [290]:
df['mate_cooking'].value_counts()

신경쓰지 않는다(불편하지 않다)    64
Name: mate_cooking, dtype: int64

In [291]:
df['mate_cooking'].isnull().sum()

8

In [292]:
df['mate_cooking'] = df['mate_cooking'].apply(lambda x : 1 if '불편하다' in str(x) else 0)
df['mate_cooking'].value_counts()

0    72
Name: mate_cooking, dtype: int64

- eat_together : 혼자 먹는 걸 선호한다, 같이 먹는 걸 선호한다

In [293]:
df['eat_together'].value_counts()

같이 먹는 걸 선호한다    43
혼자 먹는 걸 선호한다    28
그렇다              1
Name: eat_together, dtype: int64

In [294]:
df['eat_together'].isnull().sum()

0

In [295]:
df['eat_together'] = df['eat_together'].apply(lambda x : 1 if '같이 먹는 걸 선호한다' in str(x) else 0)
df['eat_together'].value_counts()

1    43
0    29
Name: eat_together, dtype: int64

- share_item : 말 안하고 사용해도 괜찮다, 사용하기 전 허락을 받으면 괜찮다, 괜찮지 않다(안된다)

In [296]:
df['share_item'].value_counts()

사용하기 전 허락을 받으면 괜찮다    64
괜찮지 않다(안된다)            5
말 안하고 사용해도 괜찮다         2
Name: share_item, dtype: int64

In [297]:
df['share_item'].isnull().sum()

1

In [298]:
def func(x):
  if '말 안하고 사용해도 괜찮다' in str(x):
    return 1
  elif '사용하기 전 허락을 받으면 괜찮다'in str(x):
    return 0.75
  elif '괜찮지 않다(안된다)' in str(x):
    return 0
  else:
    return np.nan

df['share_item'] = df['share_item'].apply(lambda x : func(x))
df['share_item'].value_counts()

0.75    64
0.00     5
1.00     2
Name: share_item, dtype: int64

In [299]:
# mean으로 결측치 채움
df['share_item'] = df['share_item'].fillna(df['share_item'].mean())
df['share_item'].value_counts()

0.750000    64
0.000000     5
1.000000     2
0.704225     1
Name: share_item, dtype: int64

- mate_alcohol : 메이트의 술 문제는 신경쓰지 않는다, 메이트가 술 냄새를 풍기는 것은 싫다, 메이트가 술을 많이 마신 것 같은 모습을 보이면 싫다, 메이트가 술주정만 하지 않으면 된다


In [300]:
print(df['mate_alcohol'].value_counts())
print("NaN : ", df['mate_alcohol'].isnull().sum())

메이트가 술주정만 하지 않으면 된다                                                         41
메이트의 술 문제는 신경쓰지 않는다                                                          9
메이트의 술 문제는 신경쓰지 않는다\t메이트가 술주정만 하지 않으면 된다                                     6
메이트가 술 냄새를 풍기는 것은 싫다\t메이트가 술을 많이 마신 것 같은 모습을 보이면 싫다                          5
메이트가 술 냄새를 풍기는 것은 싫다                                                         5
메이트가 술을 많이 마신 것 같은 모습을 보이면 싫다                                                2
소음은 없었으면 좋겠다.                                                                1
메이트가 술 냄새를 풍기는 것은 싫다\t메이트가 술을 많이 마신 것 같은 모습을 보이면 싫다\t메이트가 술주정만 하지 않으면 된다     1
메이트의 술 문제는 신경쓰지 않는다\t메이트가 술 냄새를 풍기는 것은 싫다\t메이트가 술을 많이 마신 것 같은 모습을 보이면 싫다     1
Name: mate_alcohol, dtype: int64
NaN :  1


In [301]:
def func(x):
  if '메이트가 술 냄새를 풍기는 것은 싫다' in str(x):
    return 1
  elif '메이트가 술을 많이 마신 것 같은 모습을 보이면 싫다'in str(x):
    return 0.66
  elif '메이트가 술주정만 하지 않으면 된다' in str(x):
    return 0.33
  elif '메이트의 술 문제는 신경쓰지 않는다' in str(x):
    return 0
  else:
    return np.nan

df['mate_alcohol'] = df['mate_alcohol'].apply(lambda x : func(x))
print(df['mate_alcohol'].value_counts())
print("NaN : ", df['mate_alcohol'].isnull().sum())

0.33    47
1.00    12
0.00     9
0.66     2
Name: mate_alcohol, dtype: int64
NaN :  2


In [302]:
# mean으로 결측치 채움
df['mate_alcohol'] = df['mate_alcohol'].fillna(df['mate_alcohol'].mean())
print(df['mate_alcohol'].value_counts())
print("NaN : ", df['mate_alcohol'].isnull().sum())

0.330000    47
1.000000    12
0.000000     9
0.411857     2
0.660000     2
Name: mate_alcohol, dtype: int64
NaN :  0


- mate_clean : 내 공간만 아니면 신경쓰이지 않는다, 공유하는 공간의 경우 신경쓰인다, 메이트의 공간까지 신경쓰인다

In [303]:
print(df['mate_clean'].value_counts())
print("NaN : ", df['mate_clean'].isnull().sum())

공유하는 공간의 경우 신경쓰인다      50
내 공간만 아니면 신경쓰이지 않는다    16
메이트의 공간까지 신경쓰인다         5
2                       1
Name: mate_clean, dtype: int64
NaN :  0


In [304]:
def func(x):
  if '메이트의 공간까지 신경쓰인다' in str(x):
    return 1
  elif '공유하는 공간의 경우 신경쓰인다'in str(x):
    return 0.5
  elif '내 공간만 아니면 신경쓰이지 않는다' in str(x):
    return 0
  else:
    return np.nan

df['mate_clean'] = df['mate_clean'].apply(lambda x : func(x))
print(df['mate_clean'].value_counts())
print("NaN : ", df['mate_clean'].isnull().sum())

0.5    50
0.0    16
1.0     5
Name: mate_clean, dtype: int64
NaN :  1


In [305]:
# mean으로 결측치 채움
df['mate_clean'] = df['mate_clean'].fillna(df['mate_clean'].mean())
print(df['mate_clean'].value_counts())
print("NaN : ", df['mate_clean'].isnull().sum())

0.500000    50
0.000000    16
1.000000     5
0.422535     1
Name: mate_clean, dtype: int64
NaN :  0


- permission_to_enter : 본인이 없을 때 말없이 지인이 잠깐 출입해도 상관없다., 미리 말만 하면 지인이 출입해도 상관없다., 누구든 데려오면 안된다.

In [306]:
print(df['permission_to_enter'].value_counts())
print("NaN : ", df['permission_to_enter'].isnull().sum())

미리 말만 하면 지인이 출입해도 상관없다.           56
누구든 데려오면 안된다.                     10
본인이 없을 때 말없이 지인이 잠깐 출입해도 상관없다.     6
Name: permission_to_enter, dtype: int64
NaN :  0


In [307]:
def func(x):
  if '누구든 데려오면 안된다.' in str(x):
    return 1
  elif '미리 말만 하면 지인이 출입해도 상관없다.'in str(x):
    return 0.5
  elif '본인이 없을 때 말없이 지인이 잠깐 출입해도 상관없다.' in str(x):
    return 0
  else:
    return np.nan

df['permission_to_enter'] = df['permission_to_enter'].apply(lambda x : func(x))
print(df['permission_to_enter'].value_counts())
print("NaN : ", df['permission_to_enter'].isnull().sum())

0.5    56
1.0    10
0.0     6
Name: permission_to_enter, dtype: int64
NaN :  0


### 데이터 전처리 후 테이블 확인

In [308]:
df.columns

Index(['time', 'sex', 'age', 'mbti', 'user_smoking', 'mate_smoking',
       'user_pet', 'user_bug_killer', 'mate_bug_killer', 'user_cooking',
       'mate_cooking', 'eat_together', 'share_item', 'mate_alcohol',
       'mate_clean', 'permission_to_enter', 'user_pet_dog', 'user_pet_cat',
       'user_pet_reptie_fish', 'user_pet_rodent', 'user_pet_bird', 'mate_pet',
       'mate_pet_dog', 'mate_pet_cat', 'mate_pet_reptie_fish',
       'mate_pet_rodent', 'mate_pet_bird', 'air_like_airconditioner',
       'air_night_airconditioner', 'air_like_heater', 'noise_talking',
       'noise_music', 'noise_alarm'],
      dtype='object')

In [309]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   time                      72 non-null     datetime64[ns]
 1   sex                       72 non-null     int64         
 2   age                       72 non-null     float64       
 3   mbti                      72 non-null     object        
 4   user_smoking              72 non-null     int64         
 5   mate_smoking              72 non-null     float64       
 6   user_pet                  72 non-null     int64         
 7   user_bug_killer           72 non-null     float64       
 8   mate_bug_killer           72 non-null     int64         
 9   user_cooking              72 non-null     int64         
 10  mate_cooking              72 non-null     int64         
 11  eat_together              72 non-null     int64         
 12  share_item              

In [310]:
df.head()

,time,sex,age,mbti,user_smoking,mate_smoking,user_pet,user_bug_killer,mate_bug_killer,user_cooking,mate_cooking,eat_together,share_item,mate_alcohol,mate_clean,permission_to_enter,user_pet_dog,user_pet_cat,user_pet_reptie_fish,user_pet_rodent,user_pet_bird,mate_pet,mate_pet_dog,mate_pet_cat,mate_pet_reptie_fish,mate_pet_rodent,mate_pet_bird,air_like_airconditioner,air_night_airconditioner,air_like_heater,noise_talking,noise_music,noise_alarm
0,2021-10-20 11:29:52,1,0.4,infp,0,0.0,0,0.573944,1,0,0,0,0.704225,0.411857,0.422535,1.0,0,0,0,0,0,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0.5,0.0,0.0,0
1,2021-10-22 01:16:34,0,0.6,intp,0,0.0,0,0.750000,0,0,0,0,0.750000,0.330000,0.000000,0.5,0,0,0,0,0,0.5,0.5,0.5,0.0,0.0,0.0,1,0,0.0,0.0,0.0,0
2,2021-10-22 01:29:18,1,0.2,esfp,0,0.0,0,0.000000,1,1,0,0,0.750000,0.330000,0.500000,0.5,0,0,0,0,0,0.5,0.5,0.5,0.0,0.0,0.5,1,0,1.0,0.0,0.5,1
3,2021-10-22 01:31:02,0,0.2,모른다,0,0.0,0,1.000000,0,0,0,1,0.750000,0.000000,0.500000,0.5,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0.5,0.0,0.0,0
4,2021-10-22 01:33:35,0,0.2,intp,0,0.0,0,0.750000,0,0,0,0,0.000000,1.000000,0.500000,0.5,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,1


### KNN

- KNN 알고리즘 직접 구현
- 참고 : https://iludaslab.tistory.com/36

In [311]:
# 사용자 사이의 거리를 구하는 함수
def distance(u1, u2):
  ''' 사용자 사이의 거리 리턴 '''
  return np.sqrt(np.sum(np.power((u2-u1),2)))

In [312]:
# 사용자 사이의 거리 구하기 테스트
u1 = np.array([1,3,5,6])
u2 = np.array([2,6,8,4])

print(distance(u1, u2))

4.795831523312719


In [313]:
# 일부 컬럼만 가져온 사용자 매칭 정보 데이터세트
dataset = df[['time', 'sex', 'age', 'air_night_airconditioner', 'noise_alarm', 'eat_together', 'share_item', 'mate_alcohol', 'mate_clean', 'permission_to_enter']]

distance_result = np.zeros(len(dataset))

# 기준 사용자와 다른 사용자 간의 거리를 구한다.
for i in range(len(dataset)):
  distance_result[i] = distance(dataset.iloc[0, 1:], dataset.iloc[i, 1:])

print(distance_result)

[0.         1.21545544 1.14227698 1.5708986  1.77139667 1.73631699
 1.8422834  1.67825604 1.50492415 0.55208396 1.50492415 1.67825604
 1.12463181 1.50492415 1.533055   1.50492415 1.47896312 1.96303566
 1.85062063 1.68755627 1.51815569 1.12463181 0.7169436  1.71206599
 1.00737118 1.56119567 1.14227698 0.79514872 1.61369394 1.2775999
 1.81790998 1.17906065 1.61005015 1.33984596 0.69089212 1.56119567
 1.49242484 0.41808696 1.80687484 2.01365258 1.56119567 1.66253064
 0.51458401 1.75825849 1.79647764 0.800161   1.51815569 1.50492415
 1.86218217 1.80687484 1.17621573 1.51815569 1.19888778 1.80687484
 1.80687484 1.73631699 1.32815968 1.12463181 1.14227698 1.12463181
 1.62488696 1.19487339 1.83139513 1.18068122 1.21545544 1.02703296
 1.21149594 1.50492415 1.14227698 1.60863993 1.14227698 0.90194735]


In [314]:
# 오름차순으로 정렬하여 인덱스 리턴
print(np.argsort(distance_result))

[ 0 37 42  9 34 22 27 45 71 24 65 21 12 59 57 26  2 58 70 68 50 31 63 61
 52 66  1 64 29 56 33 16 36 13 67 15  8 10 47 46 20 51 14 40 35 25  3 69
 32 28 60 41 11  7 19 23 55  5 43  4 44 53 54 38 49 30 62  6 18 48 17 39]


In [315]:
# 일부 컬럼만 가져온 사용자 매칭 정보 데이터세트
dataset = df[['mbti', 'sex', 'age', 'air_night_airconditioner', 'noise_alarm', 'eat_together', 'share_item', 'mate_alcohol', 'mate_clean', 'permission_to_enter']]

In [316]:
user = dataset.iloc[0]

In [317]:
user_data = df.iloc[0]

In [318]:
if user_data['mate_pet'] == 1:
  is_not_user_pet = df['user_pet'] == 0
  dataset = dataset[is_not_user_pet]
elif user_data['mate_pet'] == 0.5:
  # user[['pet_dog', 'pet_cat', 'pet_reptie_fish', 'pet_rodent', 'pet_bird']] = user_data[['mate_pet_dog', 'mate_pet_cat', 'mate_pet_reptie_fish', 'mate_pet_rodent', 'mate_pet_bird']]
  user['pet_dog'], user['pet_cat'], user['pet_reptie_fish'], user['pet_rodent'], user['pet_bird'] = user_data['mate_pet_dog'], user_data['mate_pet_cat'], user_data['mate_pet_reptie_fish'], user_data['mate_pet_rodent'], user_data['mate_pet_bird']
  dataset[['pet_dog', 'pet_cat', 'pet_reptie_fish', 'pet_rodent', 'pet_bird']] = df[['user_pet_dog', 'user_pet_cat', 'user_pet_reptie_fish', 'user_pet_rodent', 'user_pet_bird']]

print(user)
print(dataset)

mbti                            infp
sex                                1
age                              0.4
air_night_airconditioner           0
noise_alarm                        0
eat_together                       0
share_item                  0.704225
mate_alcohol                0.411857
mate_clean                  0.422535
permission_to_enter                1
pet_dog                          0.5
pet_cat                          0.5
pet_reptie_fish                    0
pet_rodent                         0
pet_bird                           0
Name: 0, dtype: object
    mbti  sex  age  ...  pet_reptie_fish  pet_rodent  pet_bird
0   infp    1  0.4  ...                0           0         0
1   intp    0  0.6  ...                0           0         0
2   esfp    1  0.2  ...                0           0         0
3    모른다    0  0.2  ...                0           0         0
4   intp    0  0.2  ...                0           0         0
..   ...  ...  ...  ...              ...    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  se

In [319]:
if user_data['mate_smoking'] != 0.5:
  user['smoking'] = df.iloc[0]['mate_smoking']
  dataset['smoking'] = df['user_smoking']
print(user)
print(dataset)

mbti                            infp
sex                                1
age                              0.4
air_night_airconditioner           0
noise_alarm                        0
eat_together                       0
share_item                  0.704225
mate_alcohol                0.411857
mate_clean                  0.422535
permission_to_enter                1
pet_dog                          0.5
pet_cat                          0.5
pet_reptie_fish                    0
pet_rodent                         0
pet_bird                           0
smoking                            0
Name: 0, dtype: object
    mbti  sex  age  ...  pet_rodent  pet_bird  smoking
0   infp    1  0.4  ...           0         0        0
1   intp    0  0.6  ...           0         0        0
2   esfp    1  0.2  ...           0         0        0
3    모른다    0  0.2  ...           0         0        0
4   intp    0  0.2  ...           0         0        0
..   ...  ...  ...  ...         ...       ...      ...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This

In [320]:
if user_data['air_like_airconditioner'] != 0:
  user['air_like_airconditioner'] = df.iloc[0]['air_like_airconditioner']
  dataset['air_like_airconditioner'] = df['air_like_airconditioner']
print(user)
print(dataset)

mbti                            infp
sex                                1
age                              0.4
air_night_airconditioner           0
noise_alarm                        0
eat_together                       0
share_item                  0.704225
mate_alcohol                0.411857
mate_clean                  0.422535
permission_to_enter                1
pet_dog                          0.5
pet_cat                          0.5
pet_reptie_fish                    0
pet_rodent                         0
pet_bird                           0
smoking                            0
Name: 0, dtype: object
    mbti  sex  age  ...  pet_rodent  pet_bird  smoking
0   infp    1  0.4  ...           0         0        0
1   intp    0  0.6  ...           0         0        0
2   esfp    1  0.2  ...           0         0        0
3    모른다    0  0.2  ...           0         0        0
4   intp    0  0.2  ...           0         0        0
..   ...  ...  ...  ...         ...       ...      ...


In [321]:
if user_data['air_like_heater'] != 0.5:
  user['air_like_heater'] = df.iloc[0]['air_like_heater']
  dataset['air_like_heater'] = df['air_like_heater']
print(user)
print(dataset)

mbti                            infp
sex                                1
age                              0.4
air_night_airconditioner           0
noise_alarm                        0
eat_together                       0
share_item                  0.704225
mate_alcohol                0.411857
mate_clean                  0.422535
permission_to_enter                1
pet_dog                          0.5
pet_cat                          0.5
pet_reptie_fish                    0
pet_rodent                         0
pet_bird                           0
smoking                            0
Name: 0, dtype: object
    mbti  sex  age  ...  pet_rodent  pet_bird  smoking
0   infp    1  0.4  ...           0         0        0
1   intp    0  0.6  ...           0         0        0
2   esfp    1  0.2  ...           0         0        0
3    모른다    0  0.2  ...           0         0        0
4   intp    0  0.2  ...           0         0        0
..   ...  ...  ...  ...         ...       ...      ...


In [322]:
if user_data['noise_talking'] != 0:
  user['noise_talking'] = df.iloc[0]['noise_talking']
  dataset['noise_talking'] = df['noise_talking']
print(user)
print(dataset)

mbti                            infp
sex                                1
age                              0.4
air_night_airconditioner           0
noise_alarm                        0
eat_together                       0
share_item                  0.704225
mate_alcohol                0.411857
mate_clean                  0.422535
permission_to_enter                1
pet_dog                          0.5
pet_cat                          0.5
pet_reptie_fish                    0
pet_rodent                         0
pet_bird                           0
smoking                            0
Name: 0, dtype: object
    mbti  sex  age  ...  pet_rodent  pet_bird  smoking
0   infp    1  0.4  ...           0         0        0
1   intp    0  0.6  ...           0         0        0
2   esfp    1  0.2  ...           0         0        0
3    모른다    0  0.2  ...           0         0        0
4   intp    0  0.2  ...           0         0        0
..   ...  ...  ...  ...         ...       ...      ...


In [323]:
if user_data['noise_music'] != 0:
  user['noise_music'] = df.iloc[0]['noise_music']
  dataset['noise_music'] = df['noise_music']
print(user)
print(dataset)

mbti                            infp
sex                                1
age                              0.4
air_night_airconditioner           0
noise_alarm                        0
eat_together                       0
share_item                  0.704225
mate_alcohol                0.411857
mate_clean                  0.422535
permission_to_enter                1
pet_dog                          0.5
pet_cat                          0.5
pet_reptie_fish                    0
pet_rodent                         0
pet_bird                           0
smoking                            0
Name: 0, dtype: object
    mbti  sex  age  ...  pet_rodent  pet_bird  smoking
0   infp    1  0.4  ...           0         0        0
1   intp    0  0.6  ...           0         0        0
2   esfp    1  0.2  ...           0         0        0
3    모른다    0  0.2  ...           0         0        0
4   intp    0  0.2  ...           0         0        0
..   ...  ...  ...  ...         ...       ...      ...


In [324]:
if user_data['mate_bug_killer'] != 0:
  user['bug_killer'] = df.iloc[0]['mate_bug_killer']
  dataset['bug_killer'] = df['user_bug_killer']
print(user)
print(dataset)

mbti                            infp
sex                                1
age                              0.4
air_night_airconditioner           0
noise_alarm                        0
eat_together                       0
share_item                  0.704225
mate_alcohol                0.411857
mate_clean                  0.422535
permission_to_enter                1
pet_dog                          0.5
pet_cat                          0.5
pet_reptie_fish                    0
pet_rodent                         0
pet_bird                           0
smoking                            0
bug_killer                         1
Name: 0, dtype: object
    mbti  sex  age  ...  pet_bird  smoking  bug_killer
0   infp    1  0.4  ...         0        0    0.573944
1   intp    0  0.6  ...         0        0    0.750000
2   esfp    1  0.2  ...         0        0    0.000000
3    모른다    0  0.2  ...         0        0    1.000000
4   intp    0  0.2  ...         0        0    0.750000
..   ...  ...  ...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This

In [325]:
if user_data['mate_cooking'] != 0:
  user['cooking'] = df.iloc[0]['mate_cooking']
  dataset['cooking'] = df['user_cooking']
print(user)
print(dataset)

mbti                            infp
sex                                1
age                              0.4
air_night_airconditioner           0
noise_alarm                        0
eat_together                       0
share_item                  0.704225
mate_alcohol                0.411857
mate_clean                  0.422535
permission_to_enter                1
pet_dog                          0.5
pet_cat                          0.5
pet_reptie_fish                    0
pet_rodent                         0
pet_bird                           0
smoking                            0
bug_killer                         1
Name: 0, dtype: object
    mbti  sex  age  ...  pet_bird  smoking  bug_killer
0   infp    1  0.4  ...         0        0    0.573944
1   intp    0  0.6  ...         0        0    0.750000
2   esfp    1  0.2  ...         0        0    0.000000
3    모른다    0  0.2  ...         0        0    1.000000
4   intp    0  0.2  ...         0        0    0.750000
..   ...  ...  ...

In [326]:
distance_result = np.zeros(len(dataset))

# 기준 사용자와 다른 사용자 간의 거리를 구한다.
for i in range(1, len(dataset)):
  distance_result[i] = distance(user, dataset.iloc[i, 1:])

print(distance_result)

[0.         1.42822684 1.67475273 1.72270787 1.92362838 1.89137429
 1.98909732 1.83821744 1.9403084  1.34342722 1.9403084  2.07762926
 1.35177539 1.66276778 1.6882706  1.68145672 1.92024267 2.10143023
 1.98110997 1.84671226 1.69330939 1.66276778 1.41915754 2.10503443
 1.25590474 1.98427113 1.69330939 1.46022652 1.76181955 1.48147274
 2.19198465 1.39738471 2.25660397 1.51498753 1.01972149 1.99995798
 1.65146357 1.63548057 1.9563478  2.13419697 1.73200228 2.06494749
 0.90955852 2.14277225 1.94674906 1.46296194 1.96654436 1.9403084
 2.2288388  1.9403084  1.69808228 1.69330939 1.41415413 1.9403084
 1.9563478  1.87477911 1.52528952 1.32845651 1.36649065 1.35177539
 1.77207721 1.38842444 2.20318137 1.70117845 1.40617635 1.59837314
 1.74075341 1.68145672 1.34342722 1.75719163 1.67475273 1.52102236]


In [327]:
# 오름차순으로 정렬하여 인덱스 리턴
print(np.argsort(distance_result))

[ 0 42 34 24 57  9 68 12 59 58 61 31 64 52 22  1 27 45 29 33 71 56 65 37
 36 13 21  2 70 67 15 14 51 26 20 50 63  3 40 66 69 28 60  7 19 55  5 16
  4 49 10  8 53 47 44 54 38 46 18 25  6 35 41 11 17 23 39 43 30 62 48 32]


- mbti

In [328]:
df['mbti'].value_counts()

infp    8
isfj    8
enfj    8
isfp    7
esfp    6
모른다     5
intp    5
esfj    4
istp    4
enfp    4
infj    4
intj    3
istj    2
entp    2
entj    2
Name: mbti, dtype: int64

In [340]:
MBTI_DISTANCE = np.array([
    [0.25, 0.25, 0.25, 0, 0.25, 0, 0.25, 0.25, 1, 1, 1, 1, 1, 1, 1, 1, 0.5],
    [0.25, 0.25, 0, 0.25, 0, 0.25, 0.25, 0.25, 1, 1, 1, 1, 1, 1, 1, 1, 0.5],
    [0.25, 0, 0.25, 0.25, 0.25, 0.25, 0.25, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0.5],
    [0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0, 1, 1, 1, 1, 1, 1, 1, 0.5],
    [0.25, 0, 0.25, 0.25, 0.25, 0.25, 0.25, 0, 0.5, 0.5, 0.5, 0.5, 0.75, 0.75, 0.75, 0.75, 0.5],
    [0, 0.25, 0.25, 0.25, 0.25, 0.25, 0, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
    [0.25, 0.25, 0.25, 0.25, 0.25, 0, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 0.75, 0.75, 0.75, 0, 0.5],
    [0.25, 0.25, 0, 0.25, 0, 0.25, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 0.75, 0.75, 0.75, 0.75, 0.5],
    [1, 1, 1, 0, 0.5, 0.5, 0.5, 0.5, 0.75, 0.75, 0.75, 0.75, 0.5, 0, 0.5, 0, 0.5],
    [1, 1, 1, 1, 0.5, 0.5, 0.5, 0.5, 0.75, 0.75, 0.75, 0.75, 0, 0.5, 0, 0.5, 0.5],
    [1, 1, 1, 1, 0.5, 0.5, 0.5, 0.5, 0.75, 0.75, 0.75, 0.75, 0.5, 0, 0.5, 0, 0.5],
    [1, 1, 1, 1, 0.5, 0.5, 0.5, 0.5, 0.75, 0.75, 0.75, 0.75, 0, 0.5, 0, 0.5, 0.5],
    [1, 1, 1, 1, 0.75, 0.5, 0.75, 0.75, 0.5, 0, 0.5, 0, 0.25, 0.25, 0.25, 0.25, 0.5],
    [1, 1, 1, 1, 0.75, 0.5, 0.75, 0.75, 0, 0.5, 0, 0.5, 0.25, 0.25, 0.25, 0.25, 0.5],
    [1, 1, 1, 1, 0.75, 0.5, 0.75, 0.75, 0.5, 0, 0.5, 0, 0.25, 0.25, 0.25, 0.25, 0.5],
    [1, 1, 1, 1, 0.75, 0.5, 0, 0.75, 0, 0.5, 0, 0.5, 0.25, 0.25, 0.25, 0.25, 0.5], 
    [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
])

MBTI_MAP = dict(
    infp=0,
    enfp=1,
    infj=2,
    enfj=3,
    intj=4,
    entj=5,
    intp=6,
    entp=7,
    isfp=8,
    esfp=9,
    istp=10,
    estp=11,
    isfj=12,
    esfj=13,
    istj=14,
    estj=15,
    모른다=16
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [357]:
# 사용자 사이의 거리를 구하는 함수
def distance(u1, u2):
  ''' 사용자 사이의 거리 리턴 '''
  # print(u1[0], u2[0])
  # print(temp)
  # print(MBTI_DISTANCE[MBTI_MAP[u1[0]]][MBTI_MAP[u2[0]]])
  # print(MBTI_MAP[u1[0]], MBTI_MAP[u2[0]])
  mbti = MBTI_DISTANCE[MBTI_MAP[u1[0]]][MBTI_MAP[u2[0]]] ** 2
  sum_ = np.sum(np.power((u2[1:]-u1[1:]),2))
  # print(mbti, sum_)
  return np.sqrt(sum_ + mbti)

In [358]:
distance_result = np.zeros(len(dataset))

# 기준 사용자와 다른 사용자 간의 거리를 구한다.
for i in range(1, len(dataset)):
  distance_result[i] = distance(user, dataset.iloc[i])

print(distance_result)

[0.         1.44994204 1.95058881 1.79380111 1.9398057  1.90782512
 1.98909732 1.83821744 1.9563478  1.67475273 2.00369576 2.09261639
 1.35177539 1.9403084  1.6882706  1.69994021 1.93644827 2.10143023
 1.99682165 2.10008242 1.96654436 1.9403084  1.7360899  2.10503443
 1.60539612 1.99995798 1.71166489 1.48147274 2.02583517 1.78738957
 2.40931457 1.41957178 2.27040998 1.51498753 1.42822684 1.99995798
 1.67027899 1.7102037  1.97225675 2.35686162 1.74995198 2.12461953
 1.35177539 2.36462955 1.96273582 1.484169   1.98237148 2.18284143
 2.44289222 1.9403084  1.71638674 1.96654436 1.73200228 2.18284143
 1.97225675 2.1248051  1.82387174 1.35177539 1.69330939 1.68145672
 1.789625   1.71105886 2.21732003 1.97332413 1.72549469 1.88541685
 1.74075341 1.9563478  1.67475273 1.82694346 1.69330939 1.82030465]


In [362]:
# 오름차순으로 정렬하여 인덱스 리턴
print(np.argsort(distance_result)[:20])

[ 0 42 12 57 31 34  1 27 45 33 24 36  9 68 59 14 58 70 15 37]


In [365]:
df.iloc[np.argsort(distance_result)[:20]]

,time,sex,age,mbti,user_smoking,mate_smoking,user_pet,user_bug_killer,mate_bug_killer,user_cooking,mate_cooking,eat_together,share_item,mate_alcohol,mate_clean,permission_to_enter,user_pet_dog,user_pet_cat,user_pet_reptie_fish,user_pet_rodent,user_pet_bird,mate_pet,mate_pet_dog,mate_pet_cat,mate_pet_reptie_fish,mate_pet_rodent,mate_pet_bird,air_like_airconditioner,air_night_airconditioner,air_like_heater,noise_talking,noise_music,noise_alarm
0,2021-10-20 11:29:52,1,0.4,infp,0,0.0,0,0.573944,1,0,0,0,0.704225,0.411857,0.422535,1.0,0,0,0,0,0,0.5,0.5,0.5,0.0,0.0,0.0,0,0,0.5,0.0,0.0,0
42,2021-10-22 13:48:14,1,0.4,esfj,0,0.0,0,0.750000,0,0,0,0,0.750000,0.330000,0.500000,0.5,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.5,0.5,0
12,2021-10-22 11:00:17,1,0.4,enfj,0,0.0,0,0.750000,0,0,0,0,0.750000,0.330000,0.500000,0.5,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1,0,0.0,1.0,1.0,1
57,2021-10-22 18:26:32,1,0.4,intj,0,0.0,0,1.000000,1,0,0,0,0.750000,0.330000,0.500000,0.5,0,0,0,0,0,0.5,0.0,0.5,0.0,0.0,0.0,0,0,1.0,0.0,0.0,1
31,2021-10-22 12:29:22,1,0.2,infp,0,0.0,0,0.750000,1,0,0,0,1.000000,0.330000,0.500000,0.5,0,0,0,0,0,0.5,0.5,0.5,0.0,0.0,0.0,0,1,1.0,0.0,0.0,0
34,2021-10-22 12:49:18,1,0.6,isfp,0,0.0,1,0.750000,1,1,0,0,0.750000,0.330000,0.000000,0.5,1,1,0,0,0,0.5,0.5,0.5,0.0,0.0,0.0,1,0,0.0,0.5,0.5,0
1,2021-10-22 01:16:34,0,0.6,intp,0,0.0,0,0.750000,0,0,0,0,0.750000,0.330000,0.000000,0.5,0,0,0,0,0,0.5,0.5,0.5,0.0,0.0,0.0,1,0,0.0,0.0,0.0,0
27,2021-10-22 12:02:37,1,0.2,infj,0,0.0,0,0.000000,1,0,0,0,0.750000,0.330000,1.000000,0.5,0,0,0,0,0,0.5,0.0,0.5,0.0,0.0,0.0,1,0,0.5,1.0,0.0,0
45,2021-10-22 14:08:16,1,0.2,intj,0,0.0,1,0.000000,1,0,0,0,0.750000,0.000000,0.000000,0.5,0,1,0,0,0,0.0,0.5,0.5,0.5,0.5,0.5,1,0,0.5,0.0,0.0,0
33,2021-10-22 12:40:31,1,0.2,enfj,0,0.5,0,1.000000,0,0,0,0,0.750000,0.000000,1.000000,0.5,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0,1,0.5,0.5,0.5,0
